This notebook compares the outputs from two static LAMMPS jobs one of which is an interactive job. Then outputs from the interactive and non-interactive jobs are checked for consistency.

In [1]:
from pyiron_atomistics import Project
import numpy as np
import pandas

In [2]:
pr = Project("water_interactive")
pr.remove_jobs_silently()

/tmp/ipykernel_577/1704746442.py:2: DeprecationWarning: pyiron_base.project.generic.remove_jobs_silently is deprecated: Use pr.remove_jobs(silently=True) rather than pr.remove_jobs_silently()..
  pr.remove_jobs_silently()


In [3]:
dx = 0.7
cell = np.eye(3) * 10
r_O = [0, 0, 0]
r_H1 = [dx, dx, 0]
r_H2 = [-dx, dx, 0]
water = pr.create_atoms(elements=['H', 'H', 'O'],
                        positions=[r_H1, r_H2, r_O],
                        cell=cell, pbc=True)

/tmp/ipykernel_577/2925120900.py:6: DeprecationWarning: pyiron_atomistics.project.create_atoms is deprecated: Use create.structure.atoms instead.
  water = pr.create_atoms(elements=['H', 'H', 'O'],


In [4]:
water_potential = pandas.DataFrame({    
    'Name': ['H2O_tip3p'],
    'Filename': [[]],
    'Model': ["TIP3P"],
    'Species': [['H', 'O']],
    'Config': [['# @potential_species H_O ### species in potential\n', '# W.L. Jorgensen et.al., The Journal of Chemical Physics 79, 926 (1983); https://doi.org/10.1063/1.445869\n', '#\n', '\n', 'units real\n', 'dimension 3\n', 'atom_style full\n', '\n', '# create groups ###\n', 'group O type 2\n', 'group H type 1\n', '\n', '## set charges - beside manually ###\n', 'set group O charge -0.830\n', 'set group H charge 0.415\n', '\n', '### TIP3P Potential Parameters ###\n', 'pair_style lj/cut/coul/long 10.0\n', 'pair_coeff * * 0.0 0.0 \n', 'pair_coeff 2 2 0.102 3.188 \n', 'bond_style harmonic\n', 'bond_coeff 1 450 0.9572\n', 'angle_style harmonic\n', 'angle_coeff 1 55 104.52\n', 'kspace_style pppm 1.0e-5\n', '\n']]
})

In [5]:
# Interactive job
job_int = pr.create.job.Lammps("test", delete_existing_job=True)
job_int.structure = water
job_int.potential = water_potential
job_int.interactive_open()
job_int.interactive_water_bonds = True
job_int.calc_static()
job_int.run()
job_int.interactive_close()

/srv/conda/envs/notebook/lib/python3.10/site-packages/pyiron_atomistics/lammps/base.py:241: UserWarning: Non-'metal' units are not fully supported. Your calculation should run OK, but results may not be saved in pyiron units.
  warnings.warn(


The job test was saved and received the ID: 1


[jupyter-jan-2djanssen-2ddeb-2dtegration-2dtests-2dsmvwf584:00577] mca_base_component_repository_open: unable to open mca_btl_openib: librdmacm.so.1: cannot open shared object file: No such file or directory (ignored)
/srv/conda/envs/notebook/lib/python3.10/site-packages/pylammpsmpi/wrapper/ase.py:165: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(


In [6]:
# Non-interactive job
job = pr.create.job.Lammps("test_ni", delete_existing_job=True)
job.structure = water
job.potential = water_potential
job.calc_static()
job.run()

/srv/conda/envs/notebook/lib/python3.10/site-packages/pyiron_atomistics/lammps/base.py:241: UserWarning: Non-'metal' units are not fully supported. Your calculation should run OK, but results may not be saved in pyiron units.
  warnings.warn(


The job test_ni was saved and received the ID: 2


In [7]:
# Assert that the unit converstions work even in the interactive mode

int_nodes = job_int["output/generic"].list_nodes()
usual_nodes = job["output/generic"].list_nodes()
for node in int_nodes:
    if node in usual_nodes:
        print(node)
        assert np.allclose(job_int["output/generic/" + node], job["output/generic/" + node])

cells
energy_pot
energy_tot
forces
indices
positions
pressures
steps
temperature
volume
